In [1]:
import numpy as np

# Input

In [2]:
rules = np.genfromtxt('d5_rules.txt',delimiter='|',dtype=int)
rules[:5]

array([[96, 15],
       [69, 95],
       [69, 66],
       [41, 61],
       [41, 26]])

In [3]:
test_rules = np.genfromtxt('d5_test_rules.txt',delimiter='|',dtype=int)
test_rules[:5]

array([[47, 53],
       [97, 13],
       [97, 61],
       [97, 47],
       [75, 29]])

In [4]:
rules.shape

(1176, 2)

In [5]:
d5_input = open('d5_input.txt')
d5_input = d5_input.readlines().copy()
d5_input[:5]

['27,51,67,91,42,86,58,66,39,72,96\n',
 '88,64,61,72,82,46,36,63,41,95,75\n',
 '17,62,58,66,79,22,39,72,19,96,65,63,12,41,25,46,64,75,11,95,99,23,61\n',
 '12,41,39,93,95,66,86,62,42,11,58\n',
 '18,92,26,48,67,83,69,42,62,58,66,79,39,19,15\n']

In [6]:
d5_array = [np.fromstring(line, sep=',', dtype=int) for line in d5_input]
d5_array[:5]

[array([27, 51, 67, 91, 42, 86, 58, 66, 39, 72, 96]),
 array([88, 64, 61, 72, 82, 46, 36, 63, 41, 95, 75]),
 array([17, 62, 58, 66, 79, 22, 39, 72, 19, 96, 65, 63, 12, 41, 25, 46, 64,
        75, 11, 95, 99, 23, 61]),
 array([12, 41, 39, 93, 95, 66, 86, 62, 42, 11, 58]),
 array([18, 92, 26, 48, 67, 83, 69, 42, 62, 58, 66, 79, 39, 19, 15])]

# Part One 

In [7]:
def breaks_rule(rule,index,line):
    #Check if page that should be in front is actually behind
    if line[index] == rule[0]:
        if np.any(line[:index] == rule[1]):
            return True
    #Check if page that should be behind is actually behind
    if line[index] == rule[1]:
        if np.any(line[index:] == rule[0]):
            return True
    else:
        return False

In [8]:
page_sum = 0
broken_arrays = []
for array in d5_array:
    rule_broken = False
    for index in range(array.shape[0]):
        
        mask = np.isin(rules,array).all(axis=1)
        filtered_rules = rules[mask]
        
        for rule in filtered_rules:
            
            if breaks_rule(rule,index,array):
                
                rule_broken = True
                break
        if rule_broken:
            break
    if rule_broken:
        broken_arrays.append(array)
        continue
    if not rule_broken:
        page_sum += array[round(array.shape[0]/2 - 0.5)]
page_sum

np.int64(4569)

# Part Two

In [9]:
def fix_rule(broken_array, rules):
    fixing_array = broken_array.copy()
    for index in range(fixing_array.shape[0]):
        for rule in rules:
            if breaks_rule(rule,index,fixing_array):
                val1 = np.where(fixing_array == rule[0])[0]
                val2 = np.where(fixing_array == rule[1])[0]
                fixing_array[val1[0]],fixing_array[val2[0]] = fixing_array[val2[0]],fixing_array[val1[0]]
                
                return fix_rule(fixing_array, rules)
    return fixing_array
                

In [10]:
broken_page_sum = 0
test_broken = np.array([97,13,75,29,47])
test_rules
for broken_array in broken_arrays:
    mask = np.isin(rules,broken_array).all(axis=1)
    filtered_rules = rules[mask]
    fixed_array = fix_rule(broken_array,filtered_rules)
    broken_page_sum += fixed_array[round(fixed_array.shape[0]/2 - 0.5)]
broken_page_sum

np.int64(6456)